## Hartree Fock en el átomo de Helio


In [5]:
import numpy as np



En este laboratorio vamos a implementar una rutina que permita obtener
la energía hartree-fock del átomo de helio a través de las ecuaciones
de Roothan. Para eso vamos a expresar el orbital 1s del helio en terminos
de dos funciones bases:

\begin{equation}
\phi_1(r) = 2\alpha_1^{3/2}e^{-\alpha_1 r}Y^0_0, \qquad \phi_2(r) = 2\alpha_2^{3/2}*e^{-\alpha_2 r}Y^0_0,
\end{equation}

Los exponentes $ \alpha $ son:

\begin{equation}
\alpha_1 = 1.45 \qquad \alpha_2 = 2.91
\end{equation}

In [6]:
#alpha_1 = 1.45
#alpha_2 = 2.91

alpha_1 = 1.453
alpha_2 = 2.911

El primer paso para el calculo Hartree-Fock-Roothaan es calcular la matriz de solapamiento($S_{\mu\nu}$)
y del Hamiltoniano de core ($H^{core}_{\mu\nu}$):

Es fácil verificar que la integral de solpamiento esta dada por:

\begin{equation}
S_{12} = S_{21} = \frac{8\alpha_1^{3/2}\alpha_2^{3/2}}{(\alpha_1+\alpha_2)^3}
\end{equation}

y
\begin{equation}
S_{11} = S_{22} = 1
\end{equation}


In [7]:
S_12 = (8.0*alpha_1**(3./2.)*alpha_2**(3./2.))/(alpha_1+alpha_2)**(3.0)
S_21 = S_12
S_11 = 1
S_22 = S_11

S = np.array = ([[S_11,S_12],[S_21,S_22]])
print S
print np.linalg.inv(S)

[[1, 0.8373316183566577], [0.8373316183566577, 1]]
[[ 3.34587187 -2.8016043 ]
 [-2.8016043   3.34587187]]


Los elemento diagonales del Hamiltoniano de core estan dado por:

\begin{equation}
H_{ii} =  0.5\alpha_i^2 - 2\alpha_i 
\end{equation}

y los elementos fuera de la diagonal son:

\begin{equation}
H_{12} = H_{21} = \frac{\alpha_1^{3/2}\alpha_2^{3/2}(4\alpha_1\alpha_2 - 8\alpha_1-8\alpha_2)}{(\alpha_1 + \alpha_2)^3}
\end{equation}


In [8]:
H_11 = 0.5*alpha_1**2 - 2*alpha_1
H_22 = 0.5*alpha_2**2 - 2*alpha_2
H_12 = 4*alpha_1**(3./2.)*alpha_2**(3./2.)*(alpha_1*alpha_2 - 2.0*alpha_1 - 2.0*alpha_2)/(alpha_1+alpha_2)**3.
H_21 = H_12
H = np.array = ([[H_11,H_12],[H_21,H_22]])

print H

[[-1.8503955, -1.8832915267456327], [-1.8832915267456327, -1.5850394999999997]]


Para poder calcular la matriz de Fock, necesitamos calcular las integrales de dos electrones:

# Ejercicio 1:

Determine las integrales únicas $<\mu\lambda|\nu\sigma>$  de dos electrones para el átomo de He con dos funciones base.



In [9]:
# Integrales de dos electrones:
eri1=5./8.*alpha_1 #<11|11>
eri2=5./8.*alpha_2 #<22|22>
eri3=(alpha_1**4*alpha_2+4*alpha_1**3*alpha_2**2+4*alpha_1**2*alpha_2**3+alpha_1*alpha_2**4)/(alpha_1+alpha_2)**4 #<12|12 >
eri4=20*alpha_1**3*alpha_2**3/(alpha_1+alpha_2)**5 #<11|22>
eri5=16*alpha_1**(9./2.)*alpha_2**(3./2.)/(3*alpha_1+alpha_2)**4*((12*alpha_1+8*alpha_2)/(alpha_1+alpha_2)**2+(9.*alpha_1+alpha_2)/(2.*alpha_1**2))#<11|12>
eri6=16*alpha_2**(9./2.)*alpha_1**(3./2.)/(3*alpha_2+alpha_1)**4*((12*alpha_2+8*alpha_1)/(alpha_2+alpha_1)**2+(9.*alpha_2+alpha_1)/(2.*alpha_2**2))#<22|21>
ERI=np.empty([2,2,2,2])
    # Construción del tensor de dos electrones
ERI[0][0][0][0]=eri1
ERI[1][1][1][1]=eri2
ERI[0][0][1][1]=eri3
ERI[1][1][0][0]=eri3
ERI[0][1][0][1]=eri4
ERI[1][0][0][1]=eri4
ERI[0][1][1][0]=eri4
ERI[1][0][1][0]=eri4
ERI[0][0][0][1]=eri5
ERI[0][0][1][0]=eri5
ERI[0][1][0][0]=eri5
ERI[1][0][0][0]=eri5
ERI[0][1][1][1]=eri6
ERI[1][1][0][1]=eri6
ERI[1][0][1][1]=eri6
ERI[1][1][1][0]=eri6

print ERI


[[[[ 0.908125    0.90555707]
   [ 0.90555707  1.18448063]]

  [[ 0.90555707  0.95615818]
   [ 0.95615818  1.30001249]]]


 [[[ 0.90555707  0.95615818]
   [ 0.95615818  1.30001249]]

  [[ 1.18448063  1.30001249]
   [ 1.30001249  1.819375  ]]]]


Para poder emepzar con la iteración, debemos pasar a la base ortogonal de las funciones base y así tratar
la ecuación matricial de Hartree-Fock-Roothaan como un problema de valores propios. Procedemos a diagonalizar la matriz de solapamiento y obtener la matriz diagonal s.


\begin{equation}
\Large
\mathbf{s} = \mathbf{U^{\dagger}}\mathbf{S}\mathbf{U}
\end{equation}


Ahora que es una matriz diagonal, le podemos calcular el inverso de la raiz cuadrada: $\quad\mathbf{s^{-1/2}}$


Finalmente construimos la matriz de transformación de base $ \mathbf{S^{-1/2}} $ en 
la base original, utilizando la matriz de vecotres propios $\mathbf{U}$ de 
transformación de base.

\begin{equation}
\Large
\mathbf{S^{-1/2}} = \mathbf{U}\mathbf{s^{-1/2}}\mathbf{U^{\dagger}}
\end{equation}

In [10]:
evals, evecs = np.linalg.eig(S)

print evals
s_12 = np.linalg.inv(np.identity(2)*np.sqrt(evals))
print s_12

U = evecs
print U.dot(np.transpose(U))
S_12 = U.dot(s_12).dot(np.transpose(U))

print S_12


[ 1.83733162  0.16266838]
[[ 0.73774492  0.        ]
 [ 0.          2.47941045]]
[[ 1.  0.]
 [ 0.  1.]]
[[ 1.60857768 -0.87083277]
 [-0.87083277  1.60857768]]


Ahora podemos formar la matriz de Fock, que en la primer iteración es la matriz del Hamiltoniano
core:



\begin{equation}
\Large
\mathbf{F} = \mathbf{H^{core}}
\end{equation}

Para poder solucionar el problema de valores propios debemos transformar la matriz de Fock
a la base de orbitales ortogonales:



\begin{equation}
\Large
\mathbf{F^{'}} = \mathbf{S^{\dagger -1/2}}\mathbf{F}\mathbf{S^{-1/2}}
\end{equation}


En esta base se cumple que:



\begin{equation}
\Large
\mathbf{F^{'}C^{'}} = \mathbf{C^{'}\epsilon}
\end{equation}


Para obtener los coeficientes en la base original debemos transformar de vuelta:



\begin{equation}
\Large
\mathbf{C} = \mathbf{S^{-1/2}}\mathbf{C^{'}}
\end{equation}


In [11]:
F_init = H

def F_trans(F,S_12):
    F_p = (np.transpose(S_12)).dot(F).dot(S_12)
    return F_p
F_p = F_trans(F_init,S_12)
print F_p

[[-0.71371595 -1.48888739]
 [-1.48888739 -0.22833407]]


In [12]:
C_p = np.linalg.eig(F_p)[1]

print C_p

[[-0.76186558  0.64773516]
 [-0.64773516 -0.76186558]]


Con los coeficientes en la base original, es posible caluclar la matriz densidad. Nótese 
que la sum para construir la matriz densidad es sobre los electrones y no sobre las funciones base:


\begin{equation}
\Large
\mathbf{D} = 2\sum_{i=1}^{N/2} C_{\mu i}C_{\nu i} 
\end{equation}


In [13]:
C = S_12.dot(C_p)

In [14]:
def densityM(C):
    D=np.empty([2,2])
    for i in range(2):
        for j in range(2):
            D[i][j] = C[i][0]*C[j][0]
    return 2*D

D_init = densityM(C)
print D_init
#D_init = [[0.9583,0.4791],[0.4791, 0.2396]]
#print D

[[ 0.87503477  0.50068506]
 [ 0.50068506  0.28648637]]


Ahora podemos calcular la energía Hartree-Fock inicial:

\begin{equation}
\Large
E_{HF} = 0.5\sum_{\mu \nu}^{AO} D_{\mu \nu}(H_{\mu \nu} + F_{\mu \nu})
\end{equation}

In [15]:
def energy(H,F,D):
    e = 0.5*(np.einsum('rs,rs',D,H) + np.einsum('rs,rs',D,F))
    return e
E = []
F_0 = np.zeros((2,2))
E.append(energy(H,F_0,D_init))
print E

[-1.9795622359078635]


Con los coeficientes nuevos podemos construir la matriz de Fock inicial, esta vez utliziando las integrales de repulsión electrónica: 

\begin{equation}
\Large
D_{\lambda \sigma} = \sum_{\mu \nu}^{AO} H_{\mu \nu} + D_{\mu \nu}(<\mu\sigma|\nu\lambda> - 0.5<\mu\sigma|\lambda\nu>)
\end{equation}


In [16]:
def Fock(H,D,ERI):
    G = np.einsum('tu,rstu -> rs',D,ERI) - 0.5*np.einsum('tu,ruts -> rs',D,ERI)
    F = H + G
    return F

F = Fock(H,D_init,ERI)
print F


[[-0.79730172 -0.87930134]
 [-0.87930134 -0.05540369]]


Con esta matriz de Fock repetimos los pasos anteriores para obtener una nueva energía electrónica:

In [17]:
F = Fock(H,D_init,ERI)
F_t = F_trans(F,S_12)
C_t = np.linalg.eig(F_t)[1]
C = S_12.dot(C_t)
D = densityM(C)
F = Fock(H,D,ERI)
E.append(energy(H,F,D))
print E



[-1.9795622359078635, -2.8615011475233865]


Finalmente, iteramos hasta alcanzar convergencia.

In [18]:
i = 1
#while (i != 10):
E_thresh = 1e-8
print abs(E[i] - E[i-1])
while (float(abs(E[i] - E[i-1])) > E_thresh):
#while (i != 10 ):
    print("iteration number: "+str(i))
    print(str(E[i-1] - E[i]))
    F = Fock(H,D_init,ERI)
    F_t = F_trans(F,S_12)
    C_t = np.linalg.eig(F_t)[1]
    C = S_12.dot(C_t)
    D = densityM(C)
    F = Fock(H,D,ERI)
    E.append(energy(H,F,D))
    print energy(H,F,D)
    i += 1
else:
    print("Iteration sucessfull! The Final Energy is: "+ str(E[-1])+' hartrees')
    

0.881938911616
iteration number: 1
0.881938911616
-2.86150114752
Iteration sucessfull! The Final Energy is: -2.86150114752 hartrees
